In [3]:
#!/usr/bin/env python3
from collections import deque

def solve_029A():
    """
    Find and print all of the shortest directional-keypad sequences
    that will type '029A' on the numeric keypad.

    The numeric keypad layout is (r,c):
        7(0,0) 8(0,1) 9(0,2)
        4(1,0) 5(1,1) 6(1,2)
        1(2,0) 2(2,1) 3(2,2)
           [gap]  0(3,1) A(3,2)

    The robot arm starts on 'A' at (3,2).

    Valid moves on the directional keypad are '^','v','<','>' (move arm),
    and 'A' (press the numeric button).
    """
    # Our target code:
    target_code = "029A"

    # Numeric keypad positions:
    numeric_positions = {
        '7': (0,0), '8': (0,1), '9': (0,2),
        '4': (1,0), '5': (1,1), '6': (1,2),
        '1': (2,0), '2': (2,1), '3': (2,2),
        '0': (3,1), 'A': (3,2),
    }
    numeric_positions_rev = {v: k for k,v in numeric_positions.items()}

    # We'll define a helper for moving the arm with an arrow (if valid).
    def move_arm(pos, arrow):
        """
        pos: current numeric button (like 'A' or '5')
        arrow: one of '^','v','<','>'
        returns new button or None if invalid (would hit a gap).
        """
        (r, c) = numeric_positions[pos]
        if arrow == '^':
            r2, c2 = r-1, c
        elif arrow == 'v':
            r2, c2 = r+1, c
        elif arrow == '<':
            r2, c2 = r, c-1
        else:  # '>'
            r2, c2 = r, c+1

        # Check if (r2,c2) is valid
        if (r2,c2) in numeric_positions_rev:
            return numeric_positions_rev[(r2,c2)]
        else:
            return None  # gap or off-grid

    # We do a BFS over states: (current_button, typed_string, path).
    start = ('A', '', '')  # Arm on 'A', typed_string='', path=''
    queue = deque([start])

    # We keep track of visited states with their BFS distance,
    # so we don't revisit states with a longer path.
    visited = {( 'A', '' ): 0}

    # We'll store solutions once we first find them, and keep any other solutions of the same length.
    solutions = []
    found_best_length = None

    while queue:
        pos, typed, path = queue.popleft()
        dist = len(path)  # how many button presses so far

        # If we've typed the full code '029A', we have a solution
        if typed == target_code:
            # If it's the first solution found, record the path length
            if found_best_length is None:
                found_best_length = dist
                solutions = [path]
            elif dist == found_best_length:
                # Another solution of the same minimal length
                solutions.append(path)
            # If dist > found_best_length, ignore
            continue

        # If we already found solutions and our dist is >= found_best_length, 
        # no need to expand further from here
        if found_best_length is not None and dist >= found_best_length:
            continue

        # Try each of the 5 possible actions: '^','v','<','>','A'
        for action in ['^','v','<','>','A']:
            if action == 'A':
                # Press the current button. Check if it matches the next needed char.
                next_index = len(typed)  # which char we need next
                if next_index < len(target_code):
                    needed_char = target_code[next_index]
                    if pos == needed_char:
                        # We can type this character
                        new_typed = typed + pos
                        new_pos = pos
                        new_path = path + 'A'
                        new_state = (new_pos, new_typed)
                        # Check visited
                        if new_state not in visited or visited[new_state] > dist+1:
                            visited[new_state] = dist+1
                            queue.append((new_pos, new_typed, new_path))
                # If pos != needed_char, pressing is invalid -> do nothing
            else:
                # Move the arm
                new_pos = move_arm(pos, action)
                if new_pos is not None:
                    new_typed = typed
                    new_path = path + action
                    new_state = (new_pos, new_typed)
                    if new_state not in visited or visited[new_state] > dist+1:
                        visited[new_state] = dist+1
                        queue.append((new_pos, new_typed, new_path))

    # By BFS, 'solutions' now holds all the minimal-length sequences that type "029A".
    # The puzzle's example solution says there are exactly three of them, each length=8:
    #   <A^A>^^AvvvA, <A^A^>^AvvvA, <A^A^^>AvvvA
    # Let's print them in lexicographical order for consistency.
    solutions.sort()

    print("All shortest solutions for typing '029A':")
    for sol in solutions:
        print(sol)

if __name__ == "__main__":
    solve_029A()

All shortest solutions for typing '029A':
<A^A^^>AvvvA


In [4]:
#!/usr/bin/env python3
from collections import defaultdict, deque

def solve_029A():
    """
    Find *all* shortest directional-keypad sequences to type "029A" on the numeric keypad.

    Numeric keypad layout (r,c):
        7(0,0) 8(0,1) 9(0,2)
        4(1,0) 5(1,1) 6(1,2)
        1(2,0) 2(2,1) 3(2,2)
           [gap]  0(3,1) A(3,2)

    The robot arm starts on 'A'(3,2).
    We have 5 possible actions: '^','v','<','>','A'.
    - An arrow moves the arm if not blocked by a gap.
    - 'A' presses the current numeric button; it must match the next needed char of "029A".

    We do a BFS by layers, collecting all minimal paths.
    """

    target_code = "029A"

    # Numeric keypad positions
    numeric_positions = {
        '7': (0,0), '8': (0,1), '9': (0,2),
        '4': (1,0), '5': (1,1), '6': (1,2),
        '1': (2,0), '2': (2,1), '3': (2,2),
        '0': (3,1), 'A': (3,2),
    }
    numeric_positions_rev = {v: k for k,v in numeric_positions.items()}

    def move_arm(current_button, arrow):
        """Return the new button after moving one step with arrow, or None if invalid (gap)."""
        (r, c) = numeric_positions[current_button]
        if arrow == '^':
            nr, nc = r-1, c
        elif arrow == 'v':
            nr, nc = r+1, c
        elif arrow == '<':
            nr, nc = r, c-1
        else:  # '>'
            nr, nc = r, c+1

        if (nr, nc) in numeric_positions_rev:
            return numeric_positions_rev[(nr,nc)]
        else:
            return None  # gap or off-grid => invalid move

    # We store BFS states as (pos, typed_so_far).
    # The robot arm starts on 'A', typed_so_far = "".
    start_state = ('A', '')

    # We'll do BFS by layers. current_layer maps state -> list_of_paths that reach it.
    current_layer = defaultdict(list)
    current_layer[start_state].append("")  # at distance=0, path=""

    visited = set([start_state])  # we track states we've seen at *any* distance 
                                  # (so we don't cycle infinitely)

    # We'll gather solutions in a list once we find them
    solutions = []
    found_any = False

    distance = 0
    while current_layer and not found_any:
        next_layer = defaultdict(list)

        for (pos, typed) in current_layer:
            paths_here = current_layer[(pos, typed)]
            
            # If typed == target_code, we've found complete solutions at this BFS distance
            # (But BFS won't see typed=target_code here because we don't store those states further.)
            # We'll expand states that haven't typed all chars yet.
            if len(typed) == len(target_code):
                # Already complete, skip expansion
                continue

            # Expand each path for the next action
            needed_char = target_code[len(typed)]  # the next char needed

            for path_so_far in paths_here:
                # Try each of the 5 possible actions
                for action in ['^','v','<','>','A']:
                    if action == 'A':
                        # Press the current button => it must match needed_char to proceed
                        if pos == needed_char:
                            # We can press => typed a new char
                            new_typed = typed + pos
                            new_path = path_so_far + 'A'
                            new_state = (pos, new_typed)
                            
                            if len(new_typed) == len(target_code):
                                # We finished "029A"! This is a complete solution.
                                solutions.append(new_path)
                                found_any = True
                            else:
                                # partial progress
                                if new_state not in visited:
                                    visited.add(new_state)
                                next_layer[new_state].append(new_path)
                    else:
                        # Move the arm
                        new_pos = move_arm(pos, action)
                        if new_pos is not None:
                            new_state = (new_pos, typed)
                            new_path = path_so_far + action
                            if new_state not in visited:
                                visited.add(new_state)
                            next_layer[new_state].append(new_path)

        # If we found solutions at this distance, we won't go to distance+1
        if not found_any:
            # Move on to the next BFS distance
            current_layer = next_layer
            distance += 1

    # Now 'solutions' holds all the minimal BFS solutions. Sort them:
    solutions.sort()

    # Print them
    print("All shortest solutions for typing '029A':")
    for sol in solutions:
        print(sol)

if __name__ == "__main__":
    solve_029A()

All shortest solutions for typing '029A':
<A^A>^^AvvvA
<A^A^>^AvvvA
<A^A^^>AvvvA


In [5]:
#!/usr/bin/env python3
from collections import deque

def bfs_numeric(code):
    """
    For typing a string like '029A' on the numeric keypad:
      7(0,0) 8(0,1) 9(0,2)
      4(1,0) 5(1,1) 6(1,2)
      1(2,0) 2(2,1) 3(2,2)
          [gap] 0(3,1) A(3,2)

    The robot arm starts at 'A'(3,2).
    Valid BFS actions: '^','v','<','>','A'.
      - '^','v','<','>' move the arm if not blocked by a gap.
      - 'A' presses the current numeric button, which must match the next needed char.
    
    Returns one shortest string of '^','v','<','>','A' that types the full `code`.
    """
    numeric_positions = {
        '7': (0,0), '8': (0,1), '9': (0,2),
        '4': (1,0), '5': (1,1), '6': (1,2),
        '1': (2,0), '2': (2,1), '3': (2,2),
        '0': (3,1), 'A': (3,2)
    }
    numeric_positions_rev = {v:k for k,v in numeric_positions.items()}

    def move_arm(pos, arrow):
        (r, c) = numeric_positions[pos]
        if arrow == '^':
            nr, nc = r-1, c
        elif arrow == 'v':
            nr, nc = r+1, c
        elif arrow == '<':
            nr, nc = r, c-1
        else:  # '>'
            nr, nc = r, c+1
        if (nr, nc) in numeric_positions_rev:
            return numeric_positions_rev[(nr,nc)]
        return None  # invalid / gap

    # BFS states: (current_button, typed_so_far, path_so_far).
    start = ('A', '', '')  # start pointing at 'A', typed="", path=""
    queue = deque([start])
    visited = set([( 'A','' )])  # track states (pos, typed)

    target_len = len(code)
    
    while queue:
        pos, typed, path = queue.popleft()
        # If we've typed everything, done:
        if len(typed) == target_len:
            return path  # first found => BFS => minimal

        next_char = code[len(typed)]  # which char we need next
        for action in ['^','v','<','>','A']:
            if action == 'A':
                # Press => must match next_char
                if pos == next_char:
                    new_typed = typed + pos
                    new_state = (pos, new_typed)
                    if new_state not in visited:
                        visited.add(new_state)
                        queue.append((pos, new_typed, path + 'A'))
            else:
                # Move
                new_pos = move_arm(pos, action)
                if new_pos is not None:
                    new_state = (new_pos, typed)
                    if new_state not in visited:
                        visited.add(new_state)
                        queue.append((new_pos, typed, path + action))
    # If unreachable, return None or raise an error
    return None

def bfs_directional(target_string, start_button='A'):
    """
    For typing a string of '^','v','<','>','A' on a *directional keypad*:
         [gap](0,0)  ^(0,1)  A(0,2)
         <(1,0)      v(1,1)  >(1,2)

    By puzzle spec, the robot arm starts at 'A'(0,2) by default
    (some robots start at top-right 'A').

    Valid BFS actions: '^','v','<','>','A' 
      - '^','v','<','>': move the arm if not blocked by the gap (0,0).
      - 'A': press the current directional button => must match the next char in `target_string`.

    Returns one shortest string that reproduces `target_string`.
    """
    dir_positions = {
        '^': (0,1),
        'A': (0,2),
        '<': (1,0),
        'v': (1,1),
        '>': (1,2),
    }
    dir_positions_rev = {v:k for k,v in dir_positions.items()}

    def move_arm(pos, arrow):
        (r, c) = dir_positions[pos]
        if arrow == '^':
            nr, nc = r-1, c
        elif arrow == 'v':
            nr, nc = r+1, c
        elif arrow == '<':
            nr, nc = r, c-1
        else:  # '>'
            nr, nc = r, c+1
        if (nr,nc) in dir_positions_rev:
            return dir_positions_rev[(nr,nc)]
        return None  # invalid / gap

    target_len = len(target_string)

    # BFS states: (pos, typed_so_far, path)
    # typed_so_far = how many chars of target_string we've "pressed" so far
    start = (start_button, 0, "")
    from collections import deque
    queue = deque([start])
    visited = set([(start_button, 0)])  # track (pos, typed_count)

    while queue:
        (pos, typed_count, path) = queue.popleft()
        if typed_count == target_len:
            # done
            return path

        needed_char = target_string[typed_count]

        # Try 5 actions
        for action in ['^','v','<','>','A']:
            if action == 'A':
                # Press => must match needed_char
                if pos == needed_char:
                    new_tcount = typed_count + 1
                    new_state = (pos, new_tcount)
                    if new_state not in visited:
                        visited.add(new_state)
                        queue.append((pos, new_tcount, path + 'A'))
            else:
                # Move
                new_pos = move_arm(pos, action)
                if new_pos is not None:
                    new_state = (new_pos, typed_count)
                    if new_state not in visited:
                        visited.add(new_state)
                        queue.append((new_pos, typed_count, path + action))
    # unreachable
    return None

def compute_top_level_sequence(code):
    """
    1) BFS on numeric keypad to type `code` => r1_seq (Robot #1)
    2) BFS on directional keypad to type r1_seq => r2_seq (Robot #2)
    3) BFS on directional keypad to type r2_seq => r3_seq (Robot #3, top-level)
    Returns r3_seq.
    
    For example, if code='029A', often you'll get:
      - r1_seq = <A^A>^^AvvvA
      - r2_seq = v<<A>>^A<A>AvA<^AA>A<vAAA>^A
      - r3_seq = <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<...>
    
    (There can be multiple equally-short solutions.)
    """
    # Stage 1: Robot #1 => numeric keypad
    r1_seq = bfs_numeric(code)

    # Stage 2: Robot #2 => directional keypad, starting at top-right 'A'(0,2)
    r2_seq = bfs_directional(r1_seq, start_button='A')

    # Stage 3: Robot #3 => directional keypad, starting at top-right 'A' again
    r3_seq = bfs_directional(r2_seq, start_button='A')

    return r3_seq

def main():
    # Example usage: we want the top-level nested sequence for "029A"
    code = "029A"
    top_seq = compute_top_level_sequence(code)
    print(f"Top-level sequence for {code}:")
    print(top_seq)

    # Also, if desired, we can see the intermediate sequences:
    #   - Stage 1 BFS => numeric keypad
    #   - Stage 2 BFS => second-level keypad
    #   - etc.
    # Just call them individually:
    # r1_seq = bfs_numeric(code)
    # r2_seq = bfs_directional(r1_seq)
    # r3_seq = bfs_directional(r2_seq)
    # print("Robot#1 numeric seq:", r1_seq)
    # print("Robot#2 directional seq:", r2_seq)
    # print("Robot#3 (top-level) seq:", r3_seq)

if __name__ == "__main__":
    main()

Top-level sequence for 029A:
v<A<AA>^>AvA^<Av>A^Av<<A>^>AvA^Av<<A>^>AAv<A>A^A<A>Av<A<A>^>AAA<Av>A^A


In [6]:
len('v<A<AA>^>AvA^<Av>A^Av<<A>^>AvA^Av<<A>^>AAv<A>A^A<A>Av<A<A>^>AAA<Av>A^A')


70

In [7]:
len('v<A<AA>^>AvA^<Av>A^Av<<A>^>AvA^Av<<A>^>AAv<A>A^A<A>Av<A<A>^>AAA<Av>A^A')

70

In [8]:
len('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A')

68

In [9]:
#!/usr/bin/env python3
from collections import defaultdict, deque

def bfs_numeric_all_min_solutions(code):
    """
    STAGE 1 BFS:
    Find *all* minimal arrow/press sequences that type `code` on the numeric keypad:
      7(0,0) 8(0,1) 9(0,2)
      4(1,0) 5(1,1) 6(1,2)
      1(2,0) 2(2,1) 3(2,2)
         [gap]  0(3,1) A(3,2)

    The arm starts on 'A'(3,2).
    Valid moves: '^','v','<','>' => move if not blocked by gap, 'A' => press current button (must match next char).
    code might be something like "029A".
    
    Return: list of all minimal solutions (strings of '^','v','<','>','A').
    """

    numeric_positions = {
        '7': (0,0), '8': (0,1), '9': (0,2),
        '4': (1,0), '5': (1,1), '6': (1,2),
        '1': (2,0), '2': (2,1), '3': (2,2),
        '0': (3,1), 'A': (3,2),
    }
    numeric_positions_rev = {v:k for k,v in numeric_positions.items()}

    def move_arm(pos, arrow):
        r, c = numeric_positions[pos]
        if arrow == '^':
            nr, nc = r-1, c
        elif arrow == 'v':
            nr, nc = r+1, c
        elif arrow == '<':
            nr, nc = r, c-1
        else:  # '>'
            nr, nc = r, c+1
        if (nr, nc) in numeric_positions_rev:
            return numeric_positions_rev[(nr,nc)]
        return None

    target_len = len(code)

    # We'll do a BFS by layers, storing all partial states of the same distance.
    # state = (pos, typed_count). We also keep multiple paths that lead to this same state in the same BFS layer.
    start = ('A', 0)  # arm on 'A', typed_count=0
    current_layer = defaultdict(list)
    current_layer[start].append("")  # path = ""

    visited = set([start])  # track states we've seen at or before this BFS layer
    solutions = []
    found_solutions = False

    while current_layer and not found_solutions:
        next_layer = defaultdict(list)
        for (pos, typed_count), paths in current_layer.items():
            # If typed_count == target_len, we've typed full code => record solutions
            # (But typically we won't store fully typed states in next_layer.)
            if typed_count == target_len:
                # We have minimal solutions at this BFS depth
                for pth in paths:
                    solutions.append(pth)
                found_solutions = True
                # do NOT expand further
                continue

            # Expand each path with next possible actions
            needed_char = code[typed_count]
            for pth in paths:
                for action in ['^','v','<','>','A']:
                    if action == 'A':
                        # press => must match needed_char
                        if pos == needed_char:
                            new_typed = typed_count + 1
                            new_state = (pos, new_typed)
                            new_path = pth + 'A'
                            next_layer[new_state].append(new_path)
                    else:
                        # move
                        new_pos = move_arm(pos, action)
                        if new_pos is not None:
                            new_state = (new_pos, typed_count)
                            new_path = pth + action
                            next_layer[new_state].append(new_path)
        # If we found solutions, we won't go deeper
        if not found_solutions:
            # Mark next_layer states as visited
            new_layer_keys = list(next_layer.keys())
            for st in new_layer_keys:
                if st not in visited:
                    visited.add(st)
                else:
                    # If we've visited st before, we still must combine paths
                    # because st might have new minimal paths from this layer.
                    pass
            current_layer = next_layer

    # solutions now has all minimal BFS solutions
    return solutions

def bfs_directional_all_min_solutions(target_string, start_button='A'):
    """
    STAGE 2 or 3 BFS (generic):
    We have a directional keypad:
         (0,0) gap   (0,1) ^   (0,2) A
         (1,0) <     (1,1) v   (1,2) >

    The robot arm starts at 'start_button' (default 'A' at (0,2)),
    and we want to "type" the string of '^','v','<','>','A'.

    Return *all* minimal solutions (arrow+press sequences) to produce `target_string`.
    """

    dir_positions = {
        '^': (0,1),
        'A': (0,2),
        '<': (1,0),
        'v': (1,1),
        '>': (1,2),
    }
    dir_positions_rev = {v:k for k,v in dir_positions.items()}

    def move_arm(pos, arrow):
        r, c = dir_positions[pos]
        if arrow == '^':
            nr, nc = r-1, c
        elif arrow == 'v':
            nr, nc = r+1, c
        elif arrow == '<':
            nr, nc = r, c-1
        else:
            nr, nc = r, c+1
        if (nr,nc) in dir_positions_rev:
            return dir_positions_rev[(nr,nc)]
        return None

    target_len = len(target_string)

    start = (start_button, 0)  # pointer at 'start_button', typed_count=0
    current_layer = defaultdict(list)
    current_layer[start].append("")

    visited = set([start])
    solutions = []
    found_solutions = False

    while current_layer and not found_solutions:
        next_layer = defaultdict(list)
        for (pos, tcount), paths in current_layer.items():
            if tcount == target_len:
                # already typed full target_string => minimal solutions
                for pth in paths:
                    solutions.append(pth)
                found_solutions = True
                continue

            needed_char = target_string[tcount]
            for pth in paths:
                for action in ['^','v','<','>','A']:
                    if action == 'A':
                        # press => must match needed_char
                        if pos == needed_char:
                            new_tcount = tcount + 1
                            new_state = (pos, new_tcount)
                            new_path = pth + 'A'
                            next_layer[new_state].append(new_path)
                    else:
                        # move
                        new_pos = move_arm(pos, action)
                        if new_pos is not None:
                            new_state = (new_pos, tcount)
                            new_path = pth + action
                            next_layer[new_state].append(new_path)
        if not found_solutions:
            new_layer_keys = list(next_layer.keys())
            for st in new_layer_keys:
                if st not in visited:
                    visited.add(st)
            current_layer = next_layer

    return solutions

def compute_top_level_sequence_3stage(code):
    """
    Overall approach:
      1) Stage 1 BFS on numeric keypad => all minimal solutions typing `code`.
      2) Stage 2 BFS on directional keypad => for each stage1 solution, get all minimal solutions,
         keep only those with the global minimal length among them.
      3) Stage 3 BFS on directional keypad => for each stage2 solution, get all minimal solutions,
         keep only those with the global minimal length among them.

    Return a list of final top-level solutions (there might be multiple) that are globally minimal.
    """

    # 1) All minimal solutions for numeric keypad => call these R1_solutions
    R1_solutions = bfs_numeric_all_min_solutions(code)
    if not R1_solutions:
        return []  # No solutions

    # 2) For each R1_solution, BFS directional => gather minimal solutions
    #    but we only keep the best among them (the global minimal length).
    best_r2_length = None
    best_r2_solutions = []

    for r1 in R1_solutions:
        # BFS directional from top-right 'A'(0,2)
        r2_candidate_solutions = bfs_directional_all_min_solutions(r1, start_button='A')
        if not r2_candidate_solutions:
            continue
        # find the lengths of these solutions
        # all solutions returned by the BFS are minimal among themselves, 
        # so they all have the same length
        candidate_length = len(r2_candidate_solutions[0])
        if best_r2_length is None or candidate_length < best_r2_length:
            best_r2_length = candidate_length
            best_r2_solutions = r2_candidate_solutions
        elif candidate_length == best_r2_length:
            # we add them to the set
            best_r2_solutions += r2_candidate_solutions

    if best_r2_length is None:
        # no second-stage solutions
        return []

    # 3) For each R2_solution, BFS directional => gather minimal solutions,
    #    keep only the globally minimal length
    best_r3_length = None
    best_r3_solutions = []

    for r2 in best_r2_solutions:
        r3_candidate_solutions = bfs_directional_all_min_solutions(r2, start_button='A')
        if not r3_candidate_solutions:
            continue
        candidate_length = len(r3_candidate_solutions[0])  # they share length
        if best_r3_length is None or candidate_length < best_r3_length:
            best_r3_length = candidate_length
            best_r3_solutions = r3_candidate_solutions
        elif candidate_length == best_r3_length:
            best_r3_solutions += r3_candidate_solutions

    return best_r3_solutions


def main():
    code = "029A"

    # We'll get all minimal final top-level solutions that produce code on the numeric keypad:
    final_solutions = compute_top_level_sequence_3stage(code)

    if not final_solutions:
        print(f"No solutions found for {code}")
        return

    # Remove duplicates if they exist
    final_solutions = list(set(final_solutions))

    # Sort them for consistent output
    final_solutions.sort(key=lambda s: (len(s), s))

    # Print them
    print(f"Found {len(final_solutions)} minimal top-level solution(s) for {code}, each length={len(final_solutions[0])}:")
    for sol in final_solutions:
        print(sol)

if __name__ == "__main__":
    main()

MemoryError: 

In [ ]:
from functools import cache
inputval = ""
posi = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [None, "0", "A"],
]
arr_pads = [
    [None, "^", "A"],
    ["<", "v", ">"]
]
def get_pos(arr, code):
    for i, row in enumerate(arr):
        if code in row:
            return (i, row.index(code))
@cache
def shortest(start, end, layers):
    if start == "<" and end == ">":
        pass
    if isinstance(start, str):
        start = get_pos(arr_pads, start)
    if isinstance(end, str):
        end = get_pos(arr_pads, end)

    if layers == 0:
        return 1
    elif layers < 3:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"
        if not hori and not vert:
            return shortest("A", "A", layers - 1)
        elif not hori:
            return shortest("A", vert, layers - 1) + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + shortest(vert, "A", layers - 1)
        elif not vert:
            return shortest("A", hori, layers - 1) + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + shortest(hori, "A", layers - 1)
        else:
            if start[1] == 0:
                return shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1)
            elif end[1] == 0:
                return shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
            else:
                return min(
                    shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1),
                    shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
                )
    else:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"
        if not hori and not vert:
            return shortest("A", "A", layers - 1)
        elif not hori:
            return shortest("A", vert, layers - 1) + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + shortest(vert, "A", layers - 1)
        elif not vert:
            return shortest("A", hori, layers - 1) + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + shortest(hori, "A", layers - 1)
        else:
            if start[1] == 0 and end[0] == 3:
                return shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1)
            elif end[1] == 0 and start[0] == 3:
                return shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
            else:
                return min(
                    shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1),
                    shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
                )
for start in ["<", "^", ">", "v", "A"]:
    for end in ["<", "^", ">", "v", "A"]:
        print(start, end, shortest(start, end, 1))
score = 0
while True:
    inputval = input()
    if not inputval:
        break
    intval = int(inputval[:3])
    total = 0
    for startp, endp in zip("A" + inputval[:3], inputval):
        total += shortest(get_pos(posi, startp), get_pos(posi, endp), 3)
    print(intval, total)
    score += intval * total
print(score)